In [4]:
// =========== PackageHelper Code =========================================
// This code helps with the compilation of scripts.
//
//  Add this code to your C# script before compiling the target assembly
//  Compile the target assembly

using System.IO;
using System.Collections.Generic;
using System.IO.Compression;
using Microsoft.DotNet.Interactive;

static class PackagesHelper
{
    static HashSet<string> AlreadyCopied = new HashSet<string>();
    
    public static string NewNumberedFile(string pattern)
    {
        int num = 0;
        int MAXIMUM_FILENUM = 999999;
        
        for(int i = 0; i < MAXIMUM_FILENUM; i++)
        {
            var fileName =  pattern.Replace("*", $"{i:000000}");
            if (!AlreadyCopied.Contains(fileName)) return fileName;
        }

        return "FileNot.Found";
    }

    static string GetPathRelativeToPackages(string file, DirectoryInfo directory)
    {
        var fileName = file.Replace("/", @"\");
        var strippedRoot = fileName.Replace(directory.ToString().Replace("/", @"\"), "");
        var relativePath = Path.Combine(Trim(directory.Parent.Name), Trim(directory.Name), Trim(strippedRoot));
        return relativePath;

        string Trim(string path)
        {
            return path.Trim(Path.DirectorySeparatorChar, Path.AltDirectorySeparatorChar);
        }
    }

    public static FileInfo[] GetNugetPackages()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var directories = restoreContext.ResolvedPackageReferences.Select(r => r.PackageRoot);

        var list = new List<FileInfo>();
        foreach (var directory in directories)
        {
            foreach (var file in directory.EnumerateFiles("*.nupkg", SearchOption.AllDirectories))
            {
                // Strip off and make path relative
                list.Add(file);
            }
        }

        return list.ToArray();
    }

    // This returns the relative paths to the referenced dll's
    public static IEnumerable<string> GetAssemblyProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.AssemblyPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }

    // This returns the full paths to the referenced dll's
    public static IEnumerable<string> GetNativeProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.ProbingPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }
    
    public static IEnumerable<FileInfo> GetFilesToCopy(params FileInfo[] files)
    {
        foreach(var f in files)
        {
            if(!PackagesHelper.AlreadyCopied.Contains(f.Name))
            {
                PackagesHelper.AlreadyCopied.Add(f.Name);
                yield return f;
            }
        }
    }
}

In [5]:
#i nuget:https://dotnet.myget.org/F/dotnet-corefxlab/api/v3/index.json
#r nuget:Microsoft.ML.AutoML,0.16.0-preview
#r nuget:Microsoft.Data.DataFrame,1.0.0-e190910-1

Restore sources https://dotnet.myget.org/F/dotnet-corefxlab/api/v3/index.json

In [8]:
using System.IO;
using System.Collections.Generic;
using System.IO.Compression;
using Microsoft.DotNet.Interactive;

// Preparation with compilation
//  1:  Inject the code in the two cells above into the script.
//  2:  Initialize Dll Resolution handlers Code should execute first in the compiled script, so I guess it needs to be injected at the versy start
//  3:  The PackageHelper Code is to be used by the Compilation script, it generates a zip file, two text files with reolution information, and a list of dll's 
//      that need to be copied

var workDir = new DirectoryInfo(Path.Combine(Path.GetTempPath(), Path.GetRandomFileName()));
if (!workDir.Exists) workDir.Create();

// Assembly probing paths
var assemblyProbingPaths = new FileInfo(Path.Combine(workDir.FullName, PackagesHelper.NewNumberedFile("assemblyPaths_*.probe")));
File.WriteAllLines(assemblyProbingPaths.FullName, PackagesHelper.GetAssemblyProbingPaths());

// Native probing paths
var nativeProbingPaths = new FileInfo(Path.Combine(workDir.FullName, PackagesHelper.NewNumberedFile("nativePaths_*.probe")));
File.WriteAllLines(nativeProbingPaths.FullName, PackagesHelper.GetNativeProbingPaths());

// Make a interactive_worker_*.dll
var assembly = new FileInfo(Path.Combine(workDir.FullName, PackagesHelper.NewNumberedFile("interactive_worker_*.dll")));
File.WriteAllLines(assembly.FullName, new string [] {"Hello, World"});

var targetDir = new DirectoryInfo(@"C:\Users\codec\source\repos\AssemblyRunner\AssemblyRunner\bin\Debug\netcoreapp3.1\Work");

// Copy Files
var files = PackagesHelper.GetFilesToCopy(assemblyProbingPaths, nativeProbingPaths, assembly);
foreach(var file in files)
{
    file.CopyTo(Path.Combine(targetDir.FullName, file.Name));
}

var packages = PackagesHelper.GetFilesToCopy(PackagesHelper.GetNugetPackages());
foreach(var file in packages)
{
    file.CopyTo(Path.Combine(targetDir.FullName, file.Name));
}